In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
RAW_DIR = "raw_LIV"

raw_files = sorted([
    f for f in os.listdir(RAW_DIR)
    if f.endswith(".csv")
])

raw_files

['deviceA_LIV1.csv',
 'deviceA_LIV2.csv',
 'deviceB_LIV1.csv',
 'deviceB_LIV2.csv',
 'deviceC_LIV1.csv',
 'deviceC_LIV2.csv']

In [4]:
def load_raw_jv(filepath):
    df = pd.read_csv(
        filepath,
        sep=None,              # auto-detect delimiter
        engine="python",
        encoding="latin1"
    )
    df = df.apply(pd.to_numeric, errors="coerce")
    df = df.dropna(how="all")
    df = df.dropna(axis=1, how="all")
    return df

In [5]:
def standardize_V_J(df_raw):
    df = df_raw.iloc[:, :2].copy()
    df.columns = ["Voltage_V", "Current density_mAcm2"]
    df = df.dropna()
    return df

In [6]:
def extract_device_params(df_vj, pin_mw_cm2=100, npts=8):
    V = df_vj["Voltage_V"].to_numpy()
    J = df_vj["Current density_mAcm2"].to_numpy()

    # Sort by voltage
    idx = np.argsort(V)
    V, J = V[idx], J[idx]

    # Jsc and Voc
    Jsc = np.interp(0.0, V, J)
    Voc = np.interp(0.0, J[::-1], V[::-1])

    # Power and MPP
    P = V * J
    i_mpp = np.argmax(P)
    Pmax = P[i_mpp]

    FF = Pmax / (Voc * Jsc) if Voc != 0 and Jsc != 0 else np.nan
    PCE = (Pmax / pin_mw_cm2) * 100

    # Shunt resistance (near V = 0)
    idx_v0 = np.argsort(np.abs(V))[:npts]
    Rsh = abs(np.polyfit(J[idx_v0], V[idx_v0], 1)[0]) * 1000

    # Series resistance (near J = 0)
    idx_j0 = np.argsort(np.abs(J))[:npts]
    Rs = abs(np.polyfit(J[idx_j0], V[idx_j0], 1)[0]) * 1000

    return {
        "Voc_V": Voc,
        "Jsc_mAcm2": Jsc,
        "FF": FF,
        "PCE_percent": PCE,
        "Rs_ohm_cm2": Rs,
        "Rsh_ohm_cm2": Rsh
    }


In [7]:
os.makedirs("summary", exist_ok=True)
os.makedirs("datasets", exist_ok=True)

In [8]:
rows = []

for file in raw_files:
    print("Processing:", file)

    path = os.path.join(RAW_DIR, file)

    df_raw = load_raw_jv(path)
    df_vj = standardize_V_J(df_raw)

    params = extract_device_params(df_vj)

    # Device ID (A, B, C)
    params["Device"] = file.split("_")[0]

    # Scan direction
    if "LIV1" in file.upper():
        params["Scan"] = "Reverse"
    elif "LIV2" in file.upper():
        params["Scan"] = "Forward"
    else:
        params["Scan"] = "Unknown"

    params["File"] = file

    rows.append(params)

    # Save one-row summary
    pd.DataFrame([params]).to_csv(
        f"summary/{file.replace('.csv','')}_summary.csv",
        index=False
    )

Processing: deviceA_LIV1.csv
Processing: deviceA_LIV2.csv
Processing: deviceB_LIV1.csv
Processing: deviceB_LIV2.csv
Processing: deviceC_LIV1.csv
Processing: deviceC_LIV2.csv


In [9]:
df_day4 = pd.DataFrame(rows)
df_day4

,Voc_V,Jsc_mAcm2,FF,PCE_percent,Rs_ohm_cm2,Rsh_ohm_cm2,Device,Scan,File
0,1.113055,20.178596,0.640161,14.377952,8.304344,4474.454512,deviceA,Reverse,deviceA_LIV1.csv
1,1.104883,20.177047,0.650051,14.491762,8.245944,5106.928620,deviceA,Forward,deviceA_LIV2.csv
2,1.162480,20.154208,0.758766,17.777031,7.594258,7605.396653,deviceB,Reverse,deviceB_LIV1.csv
3,1.144383,20.051937,0.746704,17.134683,6.118376,3341.636255,deviceB,Forward,deviceB_LIV2.csv
4,1.152423,20.797109,0.767192,18.387340,5.584745,8414.256139,deviceC,Reverse,deviceC_LIV1.csv
5,1.147345,20.756446,0.771115,18.363962,4.786209,4110.186343,deviceC,Forward,deviceC_LIV2.csv


In [10]:
df_day4.to_csv(
    "datasets/perovai_devices_day4_rawLIV.csv",
    index=False
)

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
RAW_DIR = "raw_LIV"

raw_files = [
    f for f in os.listdir(RAW_DIR)
    if f.endswith(".csv")
]

raw_files

['deviceA_LIV1.csv',
 'deviceA_LIV2.csv',
 'deviceB_LIV1.csv',
 'deviceB_LIV2.csv',
 'deviceC_LIV1.csv',
 'deviceC_LIV2.csv',
 'deviceD_LIV1.csv',
 'deviceD_LIV2.csv',
 'deviceE_LIV1.csv',
 'deviceE_LIV2.csv',
 'deviceF_LIV1.csv',
 'deviceF_LIV2.csv',
 'deviceG_LIV1.csv',
 'deviceG_LIV2.csv',
 'deviceH_LIV1.csv',
 'deviceH_LIV2.csv',
 'deviceI_LIV1.csv',
 'deviceI_LIV2.csv',
 'deviceJ_LIV1.csv',
 'deviceJ_LIV2.csv',
 'deviceK_LIV1.csv',
 'deviceK_LIV2.csv',
 'deviceL_LIV1.csv',
 'deviceL_LIV2.csv']

In [3]:
def load_raw_jv(filepath):
    df = pd.read_csv(
        filepath,
        encoding="latin1",
        sep=None,
        engine="python"
    )
    df = df.apply(pd.to_numeric, errors="coerce")
    df = df.dropna(how="all")
    df = df.dropna(axis=1, how="all")
    return df

In [5]:
def standardize_V_J(df_raw):
    df = df_raw.iloc[:, :2].copy()
    df.columns = ["Voltage_V", "Current density_mAcm2"]
    df = df.dropna()
    return df

In [6]:
def extract_device_params(df_vj, pin_mw_cm2=100, npts=8):
    V = df_vj["Voltage_V"].to_numpy()
    J = df_vj["Current density_mAcm2"].to_numpy()

    # Sort by voltage
    idx = np.argsort(V)
    V, J = V[idx], J[idx]

    # Jsc and Voc
    Jsc = np.interp(0.0, V, J)
    Voc = np.interp(0.0, J[::-1], V[::-1])

    # Power and MPP
    P = V * J
    i_mpp = np.argmax(P)
    Pmax = P[i_mpp]

    FF = Pmax / (Voc * Jsc) if Voc != 0 and Jsc != 0 else np.nan
    PCE = (Pmax / pin_mw_cm2) * 100

    # Shunt resistance (near V = 0)
    idx_v0 = np.argsort(np.abs(V))[:npts]
    Rsh = abs(np.polyfit(J[idx_v0], V[idx_v0], 1)[0]) * 1000

    # Series resistance (near J = 0)
    idx_j0 = np.argsort(np.abs(J))[:npts]
    Rs = abs(np.polyfit(J[idx_j0], V[idx_j0], 1)[0]) * 1000

    return {
        "Voc_V": Voc,
        "Jsc_mAcm2": Jsc,
        "FF": FF,
        "PCE_percent": PCE,
        "Rs_ohm_cm2": Rs,
        "Rsh_ohm_cm2": Rsh
    }

In [7]:
os.makedirs("summary", exist_ok=True)
os.makedirs("datasets", exist_ok=True)

In [8]:
rows = []

for file in raw_files:
    print("Processing RAW file:", file)

    path = os.path.join(RAW_DIR, file)

    df_raw = load_raw_jv(path)
    df_vj = standardize_V_J(df_raw)

    params = extract_device_params(df_vj)
    params["Device_ID"] = file.replace(".csv", "")

    # Detect scan direction
    if "LIV1" in file.upper():
        params["Scan"] = "Reverse"
    elif "LIV2" in file.upper():
        params["Scan"] = "Forward"
    else:
        params["Scan"] = "Unknown"

    rows.append(params)

    # Save one‑row summary
    pd.DataFrame([params]).to_csv(
        f"summary/{params['Device_ID']}_summary.csv",
        index=False
    )

Processing RAW file: deviceA_LIV1.csv
Processing RAW file: deviceA_LIV2.csv
Processing RAW file: deviceB_LIV1.csv
Processing RAW file: deviceB_LIV2.csv
Processing RAW file: deviceC_LIV1.csv
Processing RAW file: deviceC_LIV2.csv
Processing RAW file: deviceD_LIV1.csv
Processing RAW file: deviceD_LIV2.csv
Processing RAW file: deviceE_LIV1.csv
Processing RAW file: deviceE_LIV2.csv
Processing RAW file: deviceF_LIV1.csv
Processing RAW file: deviceF_LIV2.csv
Processing RAW file: deviceG_LIV1.csv
Processing RAW file: deviceG_LIV2.csv
Processing RAW file: deviceH_LIV1.csv
Processing RAW file: deviceH_LIV2.csv
Processing RAW file: deviceI_LIV1.csv
Processing RAW file: deviceI_LIV2.csv
Processing RAW file: deviceJ_LIV1.csv
Processing RAW file: deviceJ_LIV2.csv
Processing RAW file: deviceK_LIV1.csv
Processing RAW file: deviceK_LIV2.csv
Processing RAW file: deviceL_LIV1.csv
Processing RAW file: deviceL_LIV2.csv


In [9]:
df_day4 = pd.DataFrame(rows)
df_day4

,Voc_V,Jsc_mAcm2,FF,PCE_percent,Rs_ohm_cm2,Rsh_ohm_cm2,Device_ID,Scan
0,1.113055,20.178596,0.640161,14.377952,8.304344e+00,4.474455e+03,deviceA_LIV1,Reverse
1,1.104883,20.177047,0.650051,14.491762,8.245944e+00,5.106929e+03,deviceA_LIV2,Forward
2,1.162480,20.154208,0.758766,17.777031,7.594258e+00,7.605397e+03,deviceB_LIV1,Reverse
3,1.144383,20.051937,0.746704,17.134683,6.118376e+00,3.341636e+03,deviceB_LIV2,Forward
4,1.152423,20.797109,0.767192,18.387340,5.584745e+00,8.414256e+03,deviceC_LIV1,Reverse
5,1.147345,20.756446,0.771115,18.363962,4.786209e+00,4.110186e+03,deviceC_LIV2,Forward
6,1.162596,20.171704,0.759476,17.810883,7.581881e+00,7.279253e+03,deviceD_LIV1,Reverse
7,1.144763,20.071712,0.748184,17.191299,6.110401e+00,3.240773e+03,deviceD_LIV2,Forward
8,1.169918,20.259536,0.755708,17.911775,8.205573e+00,8.596770e+03,deviceE_LIV1,Reverse
9,1.149273,20.152430,0.748388,17.333143,6.452673e+00,3.485021e+03,deviceE_LIV2,Forward


In [10]:
df_day4.to_csv(
    "datasets/perovai_devices_day4_rawLIV.csv",
    index=False
)